Using this paper: http://openaccess.thecvf.com/content_cvpr_2018/papers/Li_Tell_Me_Where_CVPR_2018_paper.pdf
Try to come up with an implementation of the GAIN network. (For now just the GAIN, not GAIN_ext)

While I'm not 100% sure what this consists of, I beielive it is as follows:
 1. A CNN
 2. into a fully connected Layer
 3. which outputs a classification loss
 4. which is fed into another fully connected layer
    (which is stacked together into a ReLu function??? I'm not sure what this step is in the diagram.)
 5. which outputs an Attention Map
 6. which is used to apply a soft mask onto the original image
 7. which is fed into another CNN
 8. into another fully connected layer
 9. which outputs an Attention Mining Loss
10. The Attention Mining Loss is then combined which the Classification loss
    (which is then used for backprop???)
    
    
Notes on implementation (from the paper)
- It says that in the Classifcation Stream (steps 1-4 above) that that gradients dlowing back should pass through a global average pooling layer, to obtain the weights. I'm not sure how this works exactly, but in the paper it shows a function that the 
    - weight 
        - for class c
        - of unit k
        - in the l-th layer
    - should be calulated by taking the GAP (global average pool) of the partial derivative of
        - score of class c, DIVIDED BY
        - activation (f) of unit k in the l-th layer (which is just the previous weight, no?)
- Apparently the above is NOT used for back prop. Rather the weight is the "importance" of the actiavtion map (fkl) supporting class c.
    - its says that the weights matrix obtained for all layers and units is used as a kernel to apply a 2D convolution for the activation map matrix fl, to integrate all activation maps. Which is then followed by a ReLU function (so i'm assuming this is step 4 above), which should output an attention map.
    - Well if I can get that first CNN to ouput that attention map, that would be a great first start.
    - Ac = ReLu(conv(fl, wc))
    
This is a good github repo of an implementation in PyTorch:
https://github.com/AustinDoolittle/Pytorch-Gain/blob/master/gain.py
- I kindof like keras better, so lets try keras.

In [2]:
import keras

In [3]:
# The following code is just copied over from the baseline_cnn_notebook
# (with some modification)

class ModelParameters:

    def __init__(self,
                 training_data_path,
                 num_classes=28,
                 num_epochs=5,
                 batch_size=16,
                 image_rows=1708,
                 image_cols=1708,
                 row_scale_factor=4,
                 col_scale_factor=4,
                 n_channels=3,
                 using_imagenet=False,
                 shuffle=False):
        
        self.training_data_path = training_data_path
        self.num_classes = num_classes
        # what does n_epochs mean? it seems we pass this into the "epochs"
        # parameter on the "fit_generator" method on our keras model
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.row_dimension = np.int(image_rows / row_scale_factor)
        self.col_dimension = np.int(image_cols / col_scale_factor)
        self.n_channels = n_channels
        self.shuffle = shuffle
        
        if using_imagenet:
            self.row_dimension = 224
            self.col_dimension = 224
            self.n_channels = 3


class ImagePreprocessor:

    def __init__(self, modelParameters):
        self.image_path = modelParameters.training_data_path
        self.n_channels = modelParameters.n_channels
        self.row_dimension = modelParameters.row_dimension
        self.col_dimension = modelParameters.col_dimension

    def preprocess(self, image):
        image = cv2.resize(image, (self.row_dimension, self.col_dimension))
        # image = np.reshape(image, (image.shape[0], image.shape[1], self.n_channels))
        image /= 255
        return image

    def load_image(self, image_id):
        image = cv2.imread(self.image_path + image_id + '.jpg')
        if self.n_channels == 1:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert Color to Greyscalle (1 channel)
            np_image = np.zeros(shape=(image.shape[0], image.shape[1]))
        else:
            np_image = np.zeros(shape=(image.shape[0], image.shape[1], self.n_channels))
        np_image[:,:] = image
        return np_image

class ImageBatchGenerator(keras.utils.Sequence):
    
    def __init__(self, image_ids, dataframe, model_params, image_processor):
        '''
        Writing a child implementation of keras.utils.Sequence will help us
        manage our batches of data.
        Each sequence must implement __len__ and __getitem__
        This structure guarantees that the network will only train once on each
        sample per epoch which is not the case with generators.
        
        We can use this class to instantiate training and validation generators
        that we can pass into our keras model like:
        
        training_generator = ImageBatchLoader(...)
        validation_generator = ImageBatchLoader(...)
        model.set_generators(training_generator, validation_generator)
        '''
        self.image_ids = image_ids
        self.dataframe = dataframe

        # Helper classes
        self._imageProcessor = image_processor

        # Training parameters
        self.batch_size = model_params.batch_size
        self.dimensions = (model_params.row_dimension, model_params.col_dimension)
        self.n_channels = model_params.n_channels
        self.shuffle = model_params.shuffle

        # Run on_epoch_end in _init_ to init our first image batch
        self.on_epoch_end()
        
    def on_epoch_end(self):
        '''
        Tensorflow will run this method at the end of each epoch
        So this is where we will modify our batch.
        '''
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        # Denotes the number of batchs per epoch
        return int(np.floor(len(self.image_ids) / self.batch_size))
            
    def __getitem__(self, index):
        # Get this batches indexes
        indexes = self.indexes[index * self.batch_size:(index+1) * self.batch_size]

        # Get cooresponding image Ids
        batch_image_ids = [self.image_ids[i] for i in indexes]

        # Generate one batch of data
        X, y = self.__generator(batch_image_ids)
        return X, y
    
    def __generator(self, batch_image_ids):

        def get_target_class(imageid):
            # .loc will lookup the row where the passed in statement is true
            target = self.dataframe.loc[self.dataframe.imageId == imageid]
            target = target.lesion.values[0]
            return target

        X = np.empty((self.batch_size, *self.dimensions, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        for index, imageid in enumerate(batch_image_ids):
            image = self._imageProcessor.load_image(imageid)
            image = self._imageProcessor.preprocess(image)

            X[index] = image
            y[index] = get_target_class(imageid)

        return X, y


class PredictGenerator:

    def __init__(self, image_ids, image_processor, image_path):
        self.image_processor = image_processor
        self.image_processor.image_path = image_path
        self.image_ids = image_ids

    def predict(self, model):
        y = np.empty(shape=(len(self.image_ids)))
        for n in range(len(self.image_ids)):
            image = self.image_processor.load_image(self.image_ids[n])
            image = self.image_processor.preprocess(image)
            image = image.reshape((1, *image.shape))
            y[n] = model.predict(image)
        return y

In [ ]:
class GAIN_Model:
    
    def __init__(self, model_params):
        self.img_rows = model_params.row_dimension
        self.img_cols = model_params.col_dimension
        self.n_channels = model_params.n_channels
        self.input_shape = (self.img_rows, self.img_cols, self.n_channels)
        self.num_epochs = model_params.num_epochs
        # What metrics should I use for generating the attention map?
        # The above linked PyTorch implemntation just uses Accuracy, so I'll stick with that for now.
        self.metrics = ['accuracy'] 
        
    def build_model(self):
        # Interesting, it looks like in that above linked PyTorch implementation, the classification model
        # is just passed in as an argument.

Further Reading:
- Attention Maps
    - https://jacobgil.github.io/deeplearning/class-activation-maps
    - https://github.com/jacobgil/keras-cam
    - https://github.com/tdeboissiere/VGG16CAM-keras

- Getting Gradients
    - https://www.programcreek.com/python/example/93762/keras.backend.gradients
